In [15]:
import pyspark
spark = pyspark.sql.SparkSession.builder.getOrCreate()
import pyspark.sql.functions as F
spark

In [9]:
df = spark.read.csv('case.csv', header = True, inferSchema=True)
df.show(1, vertical = True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 1/1/18 0:42          
 case_closed_date     | 1/1/18 12:29         
 SLA_due_date         | 9/26/20 0:42         
 case_late            | NO                   
 num_days_late        | -998.5087616000001   
 case_closed          | YES                  
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
only showing top 1 row



In [7]:
df.printSchema()

root
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: string (nullable = true)
 |-- case_closed_date: string (nullable = true)
 |-- SLA_due_date: string (nullable = true)
 |-- case_late: string (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: string (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: integer (nullable = true)



In [12]:
from pyspark.sql.types import StructType, StructField,StringType

schema = StructType(
    [
        StructField('sorce_id', StringType()),
        StructField('source_username', StringType())
    ]

)
schema

StructType(List(StructField(sorce_id,StringType,true),StructField(source_username,StringType,true)))

In [13]:
df.groupBy('case_late').count().show()

+---------+------+
|case_late| count|
+---------+------+
|      YES| 94503|
|       NO|747201|
+---------+------+



In [14]:
df.groupBy('case_closed').count().show()

+-----------+------+
|case_closed| count|
+-----------+------+
|        YES|823594|
|         NO| 18110|
+-----------+------+



In [22]:
df = (df.withColumn('case_closed', df.case_closed == 'YES')
   .withColumn('case_late', df.case_late == 'YES'))
df.show(1, vertical = True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 1/1/18 0:42          
 case_closed_date     | 1/1/18 12:29         
 SLA_due_date         | 9/26/20 0:42         
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
only showing top 1 row



In [26]:
df = df.withColumn('council_district', df.council_district.cast('string'))
df.printSchema()

root
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: string (nullable = true)
 |-- case_closed_date: string (nullable = true)
 |-- SLA_due_date: string (nullable = true)
 |-- case_late: boolean (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: boolean (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: string (nullable = true)



In [27]:
df.explain()

== Physical Plan ==
*(1) Project [case_id#219, case_opened_date#220, case_closed_date#221, SLA_due_date#222, ((case_late#223 = YES) = true) AS case_late#611, num_days_late#224, ((case_closed#225 = YES) = true) AS case_closed#596, dept_division#226, service_request_type#227, SLA_days#228, case_status#229, source_id#230, request_address#231, cast(council_district#232 as string) AS council_district#698]
+- *(1) FileScan csv [case_id#219,case_opened_date#220,case_closed_date#221,SLA_due_date#222,case_late#223,num_days_late#224,case_closed#225,dept_division#226,service_request_type#227,SLA_days#228,case_status#229,source_id#230,request_address#231,council_district#232] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/Users/dom/codeup_data_science/ds-methodologies-exercises/spark/case.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<case_id:int,case_opened_date:string,case_closed_date:string,SLA_due_date:string,case_late:...


In [33]:
df.count()

841704